# Imports

In [1]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

# Data Compile
Organize behavioral data from each participant into one dataframe for the whole experiment.

In [9]:
data_dirs = ['../sustained_attention_experiment/data','../variable_attention_experiment/data']
filenames = ['../parsed_data/behavioral_data_sustained.csv', '../parsed_data/behavioral_data_variable.csv']

for data,file in zip(data_dirs, filenames):
    
    sub_list = []
    
    for sub_dir in os.listdir(data):
        sub_list.append(add_level(sum_pd(data + '/' + sub_dir)))

    exp_raw = pd.concat(sub_list)
    exp_raw.to_csv(file)